# Script to calculate SEM model for the WMHOB project

In [2]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)
library(mediation)
library(lavaan)
library(lavaanPlot)
library(plotly)
library(MVN)
#install.packages('MissMech')
#library(semTools)
#library(MissMech)
#install.packages('lattice')
#library(lattice)
#install.packages("ellipse")
library('gridExtra')

# Load data

In [3]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])
colnames(di)=c('eid',colnames(di)[2:length(di)])
ukb_area <- ukb_df('ukb40807', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
glob2rx("area_of*_2_*") # Create a regexp to use in grep
ukbb_area=(ukb_area[c(1, grep('^area_of.*_2_',colnames(ukb_area)))])

### Exclude outliers in the imaging dataset (n=22k)
di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Mental health issues
di$depression <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
#di$schizophrenia[di[i]=="Schizophrenia"] <- 1
di$depression[di[i]=="Depression"] <- 1
#di$schizophrenia[di[i]=="Mania, hypomania, bipolar or manic-depression"] <- 1
#di$schizophrenia[di[i]=="Bulimia nervosa"] <- 1
#di$schizophrenia[di[i]=="Anorexia nervosa"] <- 1
#di$schizophrenia[di[i]=="Attention deficit or attention deficit and hyperactivity disorder (ADD/ADHD)"] <- 1
#di$schizophrenia[di[i]=="Panic attacks"] <- 1
}


# Exclude participants who have diabetes
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('diabetes_diagnosed_by_doctor_f2443_0_0', 
                          colnames(di)))) { #Columns with vascular diseases
di$diabetes[di[i] =="Yes"] <- 1
}

# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor_f6150_0', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i] =="High blood pressure"] <- 1
}



#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
#di$excluded[di$bipolar_dis ==1] <- 1
#di$excluded[di$personality_dis == 1] <- 1
#di$excluded[di$schizophrenia == 1] <- 1
#di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
#di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1
#di$excluded[di$diabetes ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')
ukbb_all=merge(ukbb_all, ukbb_area, by='eid',all.x=TRUE)


nrow(ukbb_all)

ukbb_all$WMH_logscaled=log(ukbb_all$total_volume_of_white_matter_hyperintensities_from_t1_and_t2_flair_images_f25781_2_0/
                    ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)
ukbb_all$logBMI=log(ukbb_all$body_mass_index_bmi_f21001_0_0)
ukbb_all$WHR=ukbb_all$waist_circumference_f48_0_0/ukbb_all$hip_circumference_f49_0_0

## Merge with white matter integrity variables
#ukb_WM <- ukb_df('ukb35375', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
#ukbb_all=merge(ukbb_all,ukb_WM,by='eid',all.x=TRUE)

ukbb_all$Hypertension=0
ukbb_all$Hypertension[ukbb_all$vascular_heart_diagnoses=='1']=1
ukbb_all$Diabetes=0
ukbb_all$Diabetes[ukbb_all$diabetes=='1']=1
ukbb_all$Depression=0
ukbb_all$Depression[ukbb_all$depression=='1']=1


PA=read.table('/dagher/dagher11/filip/OBAL/alcohol.csv',sep=',',header=T)
ukbb_all=merge(ukbb_all,PA,by='eid',all.x=T)


In [4]:
UKB_Blood <- ukb_df("ukb41240", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")

In [5]:
ukbb_all=merge(ukbb_all,UKB_Blood, by='eid', all.x=TRUE)

In [6]:
rm(UKB_Blood)

## Standardise brain volume variables

In [13]:
for (i in c(2287:2300)) {
    ukbb_all[[i]]=scale(ukbb_all[[i]]/ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)
    }

In [14]:
for (i in c(58:88, 120:150)) {
    ukbb_all[[i]]=scale(ukbb_all[[i]])
    }

In [15]:
for (i in c(89:119, 151:181)) {
    ukbb_all[[i]]=scale(ukbb_all[[i]]/ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)
    }

## Build mean GM variables

In [7]:
ukbb_all$MeanThicknessLeft=rowMeans(ukbb_all[grep('thickness',colnames(ukbb_all))][1:31])
ukbb_all$MeanThicknessRight=rowMeans(ukbb_all[grep('thickness',colnames(ukbb_all))][2:62])
ukbb_all$MeanAreaLeft=rowMeans(ukbb_all[grep('area_of',colnames(ukbb_all))][1:31])
ukbb_all$MeanAreaRight=rowMeans(ukbb_all[grep('area_of',colnames(ukbb_all))][32:62])
                                             
ukbb_all$MeanThickness=scale(((ukbb_all$MeanThicknessLeft*ukbb_all$MeanAreaLeft)+
                             (ukbb_all$MeanThicknessRight*ukbb_all$MeanAreaRight))/
                             (ukbb_all$MeanAreaLeft + ukbb_all$MeanAreaRight))                                     

In [8]:
ukbb_all$GM=scale(rowSums(ukbb_all[c(89:119, 151:181)])/
             ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)
#ukbb_all$GM=scale(ukbb_all$volume_of_peripheral_cortical_grey_matter_f25002_2_0/
#                  ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)

In [9]:
ukbb_all$GMsub=scale(ukbb_all$volume_of_subcortgray_whole_brain_f26517_2_0/
                    ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)

## Convert cognitive variables

In [10]:
ukbb_all$Mean_pairs=scale(rowMeans(ukbb_all[274:276], na.rm=TRUE, dims=1))
ukbb_all$RT=scale(rowMeans(ukbb_all[418:429], na.rm=TRUE, dims=1))
ukbb_all$Prospective_memory<-(as.numeric(factor(car::recode(ukbb_all$prospective_memory_result_f20018_2_0, 
                                         "'Correct recall on first attempt'='1'; 
                                        'Correct recall on second attempt'='2';
                                        'Instruction not recalled, either skipped or incorrect'='3'"), ordered=TRUE)))
ukbb_cog=ukbb_all


## Scale and log-transform variables

In [11]:
ukbb_all$Sex<-car::recode(ukbb_all$sex_f31_0_0, "'Male'=1; else=0")
ukbb_all$Sex=as.numeric(levels(ukbb_all$Sex))[ukbb_all$Sex]
ukbb_all$DigitSpan=scale(ukbb_all$maximum_digits_remembered_correctly_f4282_2_0)
ukbb_all$FluidIntelligence=scale(ukbb_all$fluid_intelligence_score_f20016_2_0)
ukbb_all$Puzzles=scale(ukbb_all$number_of_puzzles_correct_f21004_2_0)
ukbb_all$WHRscaled=scale(ukbb_all$WHR)
ukbb_all$CRP_scaled=(log(ukbb_all$creactive_protein_f30710_0_0))
ukbb_all$SBP=scale(rowMeans(data.frame(ukbb_all$systolic_blood_pressure_automated_reading_f4080_2_1, ukbb_all$systolic_blood_pressure_automated_reading_f4080_2_0)))
ukbb_all$TG=scale(log(ukbb_all$triglycerides_f30870_0_0))
ukbb_all$HDL=scale(log(ukbb_all$hdl_cholesterol_f30760_0_0))
ukbb_all$LDL=scale(log(ukbb_all$ldl_direct_f30780_0_0))
ukbb_all$CHOL=scale(log(ukbb_all$cholesterol_f30690_0_0))
ukbb_all$waist=scale(ukbb_all$waist_circumference_f48_0_0)
ukbb_all$hip=scale(ukbb_all$hip_circumference_f49_0_0)
ukbb_all$HbA1c=scale(log(ukbb_all$glycated_haemoglobin_hba1c_f30750_0_0))
ukbb_all$Glucose=scale(log(ukbb_all$glucose_f30740_0_0))
ukbb_all$SBP=scale(rowMeans(cbind(ukbb_all$systolic_blood_pressure_automated_reading_f4080_0_0,
                           ukbb_all$systolic_blood_pressure_automated_reading_f4080_0_1)))
ukbb_all$DBP=scale(rowMeans(cbind(ukbb_all$diastolic_blood_pressure_automated_reading_f4079_0_0,
                           ukbb_all$diastolic_blood_pressure_automated_reading_f4079_0_1)))
ukbb_all$BFP=scale(ukbb_all$body_fat_percentage_f23099_0_0)
ukbb_all$FMI=scale((ukbb_all$body_fat_percentage_f23099_0_0/100*ukbb_all$weight_f21002_0_0)/
                  (ukbb_all$height_f12144_2_0/100)^2)
ukbb_all$ApoB=scale(log(ukbb_all$apolipoprotein_b_f30640_0_0))

In [12]:
ukbb_all$Age_scaled=scale(ukbb_all$age_when_attended_assessment_centre_f21003_0_0)

In [13]:
ukbb_all$BMI=scale(log(ukbb_all$body_mass_index_bmi_f21001_0_0))

## Create one dataframe with all variables for the SEM and remove outliers

In [14]:
Lavaan_SEM=select(ukbb_all, Sex, Hypertension, Diabetes,
                  Prospective_memory, FluidIntelligence, DigitSpan, Puzzles,
                  Mean_pairs, RT,  
                  mean_thickness_of_caudalanteriorcingulate_left_hemisphere_f27174_2_0,
                    mean_thickness_of_caudalmiddlefrontal_left_hemisphere_f27175_2_0,
                    mean_thickness_of_cuneus_left_hemisphere_f27176_2_0,
                    mean_thickness_of_entorhinal_left_hemisphere_f27177_2_0,
                    mean_thickness_of_fusiform_left_hemisphere_f27178_2_0,
                    mean_thickness_of_inferiorparietal_left_hemisphere_f27179_2_0,
                    mean_thickness_of_inferiortemporal_left_hemisphere_f27180_2_0,
                    mean_thickness_of_isthmuscingulate_left_hemisphere_f27181_2_0,
                    mean_thickness_of_lateraloccipital_left_hemisphere_f27182_2_0,
                    mean_thickness_of_lateralorbitofrontal_left_hemisphere_f27183_2_0,
                    mean_thickness_of_lingual_left_hemisphere_f27184_2_0,
                    mean_thickness_of_medialorbitofrontal_left_hemisphere_f27185_2_0,
                    mean_thickness_of_middletemporal_left_hemisphere_f27186_2_0,
                    mean_thickness_of_parahippocampal_left_hemisphere_f27187_2_0,
                    mean_thickness_of_paracentral_left_hemisphere_f27188_2_0,
                    mean_thickness_of_parsopercularis_left_hemisphere_f27189_2_0,
                    mean_thickness_of_parsorbitalis_left_hemisphere_f27190_2_0,
                    mean_thickness_of_parstriangularis_left_hemisphere_f27191_2_0,
                    mean_thickness_of_pericalcarine_left_hemisphere_f27192_2_0,
                    mean_thickness_of_postcentral_left_hemisphere_f27193_2_0,
                    mean_thickness_of_posteriorcingulate_left_hemisphere_f27194_2_0,
                    mean_thickness_of_precentral_left_hemisphere_f27195_2_0,
                    mean_thickness_of_precuneus_left_hemisphere_f27196_2_0,
                    mean_thickness_of_rostralanteriorcingulate_left_hemisphere_f27197_2_0,
                    mean_thickness_of_rostralmiddlefrontal_left_hemisphere_f27198_2_0,
                    mean_thickness_of_superiorfrontal_left_hemisphere_f27199_2_0,
                    mean_thickness_of_superiorparietal_left_hemisphere_f27200_2_0,
                    mean_thickness_of_superiortemporal_left_hemisphere_f27201_2_0,
                    mean_thickness_of_supramarginal_left_hemisphere_f27202_2_0,
                    mean_thickness_of_transversetemporal_left_hemisphere_f27203_2_0,
                    mean_thickness_of_insula_left_hemisphere_f27204_2_0,
                    mean_thickness_of_caudalanteriorcingulate_right_hemisphere_f27267_2_0,
                    mean_thickness_of_caudalmiddlefrontal_right_hemisphere_f27268_2_0,
                    mean_thickness_of_cuneus_right_hemisphere_f27269_2_0,
                    mean_thickness_of_entorhinal_right_hemisphere_f27270_2_0,
                    mean_thickness_of_fusiform_right_hemisphere_f27271_2_0,
                    mean_thickness_of_inferiorparietal_right_hemisphere_f27272_2_0,
                    mean_thickness_of_inferiortemporal_right_hemisphere_f27273_2_0,
                    mean_thickness_of_isthmuscingulate_right_hemisphere_f27274_2_0,
                    mean_thickness_of_lateraloccipital_right_hemisphere_f27275_2_0,
                    mean_thickness_of_lateralorbitofrontal_right_hemisphere_f27276_2_0,
                    mean_thickness_of_lingual_right_hemisphere_f27277_2_0,
                    mean_thickness_of_medialorbitofrontal_right_hemisphere_f27278_2_0,
                    mean_thickness_of_middletemporal_right_hemisphere_f27279_2_0,
                    mean_thickness_of_parahippocampal_right_hemisphere_f27280_2_0,
                    mean_thickness_of_paracentral_right_hemisphere_f27281_2_0,
                    mean_thickness_of_parsopercularis_right_hemisphere_f27282_2_0,
                    mean_thickness_of_parsorbitalis_right_hemisphere_f27283_2_0,
                    mean_thickness_of_parstriangularis_right_hemisphere_f27284_2_0,
                    mean_thickness_of_pericalcarine_right_hemisphere_f27285_2_0,
                    mean_thickness_of_postcentral_right_hemisphere_f27286_2_0,
                    mean_thickness_of_posteriorcingulate_right_hemisphere_f27287_2_0,
                    mean_thickness_of_precentral_right_hemisphere_f27288_2_0,
                    mean_thickness_of_precuneus_right_hemisphere_f27289_2_0,
                    mean_thickness_of_rostralanteriorcingulate_right_hemisphere_f27290_2_0,
                    mean_thickness_of_rostralmiddlefrontal_right_hemisphere_f27291_2_0,
                    mean_thickness_of_superiorfrontal_right_hemisphere_f27292_2_0,
                    mean_thickness_of_superiorparietal_right_hemisphere_f27293_2_0,
                    mean_thickness_of_superiortemporal_right_hemisphere_f27294_2_0,
                    mean_thickness_of_supramarginal_right_hemisphere_f27295_2_0,
                    mean_thickness_of_transversetemporal_right_hemisphere_f27296_2_0,
                    mean_thickness_of_insula_right_hemisphere_f27297_2_0,
                    volume_of_caudalanteriorcingulate_left_hemisphere_f27205_2_0,
                    volume_of_caudalmiddlefrontal_left_hemisphere_f27206_2_0,
                    volume_of_cuneus_left_hemisphere_f27207_2_0,
                    volume_of_entorhinal_left_hemisphere_f27208_2_0,
                    volume_of_fusiform_left_hemisphere_f27209_2_0,
                    volume_of_inferiorparietal_left_hemisphere_f27210_2_0,
                    volume_of_inferiortemporal_left_hemisphere_f27211_2_0,
                    volume_of_isthmuscingulate_left_hemisphere_f27212_2_0,
                    volume_of_lateraloccipital_left_hemisphere_f27213_2_0,
                    volume_of_lateralorbitofrontal_left_hemisphere_f27214_2_0,
                    volume_of_lingual_left_hemisphere_f27215_2_0,
                    volume_of_medialorbitofrontal_left_hemisphere_f27216_2_0,
                    volume_of_middletemporal_left_hemisphere_f27217_2_0,
                    volume_of_parahippocampal_left_hemisphere_f27218_2_0,
                    volume_of_paracentral_left_hemisphere_f27219_2_0,
                    volume_of_parsopercularis_left_hemisphere_f27220_2_0,
                    volume_of_parsorbitalis_left_hemisphere_f27221_2_0,
                    volume_of_parstriangularis_left_hemisphere_f27222_2_0,
                    volume_of_pericalcarine_left_hemisphere_f27223_2_0,
                    volume_of_postcentral_left_hemisphere_f27224_2_0,
                    volume_of_posteriorcingulate_left_hemisphere_f27225_2_0,
                    volume_of_precentral_left_hemisphere_f27226_2_0,
                    volume_of_precuneus_left_hemisphere_f27227_2_0,
                    volume_of_rostralanteriorcingulate_left_hemisphere_f27228_2_0,
                    volume_of_rostralmiddlefrontal_left_hemisphere_f27229_2_0,
                    volume_of_superiorfrontal_left_hemisphere_f27230_2_0,
                    volume_of_superiorparietal_left_hemisphere_f27231_2_0,
                    volume_of_superiortemporal_left_hemisphere_f27232_2_0,
                    volume_of_supramarginal_left_hemisphere_f27233_2_0,
                    volume_of_transversetemporal_left_hemisphere_f27234_2_0,
                    volume_of_insula_left_hemisphere_f27235_2_0,
                    volume_of_caudalanteriorcingulate_right_hemisphere_f27298_2_0,
                    volume_of_caudalmiddlefrontal_right_hemisphere_f27299_2_0,
                    volume_of_cuneus_right_hemisphere_f27300_2_0,
                    volume_of_entorhinal_right_hemisphere_f27301_2_0,
                    volume_of_fusiform_right_hemisphere_f27302_2_0,
                    volume_of_inferiorparietal_right_hemisphere_f27303_2_0,
                    volume_of_inferiortemporal_right_hemisphere_f27304_2_0,
                    volume_of_isthmuscingulate_right_hemisphere_f27305_2_0,
                    volume_of_lateraloccipital_right_hemisphere_f27306_2_0,
                    volume_of_lateralorbitofrontal_right_hemisphere_f27307_2_0,
                    volume_of_lingual_right_hemisphere_f27308_2_0,
                    volume_of_medialorbitofrontal_right_hemisphere_f27309_2_0,
                    volume_of_middletemporal_right_hemisphere_f27310_2_0,
                    volume_of_parahippocampal_right_hemisphere_f27311_2_0,
                    volume_of_paracentral_right_hemisphere_f27312_2_0,
                    volume_of_parsopercularis_right_hemisphere_f27313_2_0,
                    volume_of_parsorbitalis_right_hemisphere_f27314_2_0,
                    volume_of_parstriangularis_right_hemisphere_f27315_2_0,
                    volume_of_pericalcarine_right_hemisphere_f27316_2_0,
                    volume_of_postcentral_right_hemisphere_f27317_2_0,
                    volume_of_posteriorcingulate_right_hemisphere_f27318_2_0,
                    volume_of_precentral_right_hemisphere_f27319_2_0,
                    volume_of_precuneus_right_hemisphere_f27320_2_0,
                    volume_of_rostralanteriorcingulate_right_hemisphere_f27321_2_0,
                    volume_of_rostralmiddlefrontal_right_hemisphere_f27322_2_0,
                    volume_of_superiorfrontal_right_hemisphere_f27323_2_0,
                    volume_of_superiorparietal_right_hemisphere_f27324_2_0,
                    volume_of_superiortemporal_right_hemisphere_f27325_2_0,
                    volume_of_supramarginal_right_hemisphere_f27326_2_0,
                    volume_of_transversetemporal_right_hemisphere_f27327_2_0,
                    volume_of_insula_right_hemisphere_f27328_2_0, 
                    volume_of_grey_matter_in_thalamus_left_f25878_2_0,
                    volume_of_grey_matter_in_thalamus_right_f25879_2_0,
                    volume_of_grey_matter_in_caudate_left_f25880_2_0,
                    volume_of_grey_matter_in_caudate_right_f25881_2_0,
                    volume_of_grey_matter_in_putamen_left_f25882_2_0,
                    volume_of_grey_matter_in_putamen_right_f25883_2_0,
                    volume_of_grey_matter_in_pallidum_left_f25884_2_0,
                    volume_of_grey_matter_in_pallidum_right_f25885_2_0,
                    volume_of_grey_matter_in_hippocampus_left_f25886_2_0,
                    volume_of_grey_matter_in_hippocampus_right_f25887_2_0,
                    volume_of_grey_matter_in_amygdala_left_f25888_2_0,
                    volume_of_grey_matter_in_amygdala_right_f25889_2_0,
                    volume_of_grey_matter_in_ventral_striatum_left_f25890_2_0,
                    volume_of_grey_matter_in_ventral_striatum_right_f25891_2_0,
                    GM ,GMsub, MeanThickness,
                     SBP, DBP, 
                    WMH_logscaled, WHRscaled, BMI, CRP_scaled, TG, HDL, 
                    BFP, LDL, ApoB, HbA1c, Glucose,
                    qualifications_f6138_2_0, qualifications_f6138_0_0, Depression,
                    frequency_of_drinking_alcohol_20414.0.0, 
                    number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.0.0,
                    number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0,
                    townsend_deprivation_index_at_recruitment_f189_0_0,
                    age_when_attended_assessment_centre_f21003_2_0, age_when_attended_assessment_centre_f21003_0_0,
                    average_total_household_income_before_tax_f738_0_0, 
                    average_total_household_income_before_tax_f738_2_0,recent_feelings_of_depression_f20510_0_0,
                    smoking_status_f20116_0_0, smoking_status_f20116_2_0,
                    alcohol_f100022_0_0, eid)



#Lavaan_SEM=Lavaan_SEM[complete.cases(Lavaan_SEM[1:(length(Lavaan_SEM))]),]
for (j in 5:(length(Lavaan_SEM)-16)) { # 
    Q3=as.numeric(quantile(Lavaan_SEM[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(Lavaan_SEM[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    Lavaan_SEM[j][Lavaan_SEM[j]<lower]=NA
    Lavaan_SEM[j][Lavaan_SEM[j]>upper]=NA
}

nrow(Lavaan_SEM)
Lavaan_SEM$HDL=-1*Lavaan_SEM$HDL #Because of constraining the loadings later together with TG

## Residualise variables for age, sex, qualification, income, TDI and smoking

In [15]:
for (i in c(4:(length(Lavaan_SEM)-25))){
    
    Lavaan_SEM[[i]]=resid(lm(Lavaan_SEM[[i]] ~ age_when_attended_assessment_centre_f21003_2_0
                                 + average_total_household_income_before_tax_f738_2_0 + Sex
                                 + Depression + frequency_of_drinking_alcohol_20414.0.0
                                 + number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0 
                                 + smoking_status_f20116_2_0 + qualifications_f6138_2_0 + 
                                 + townsend_deprivation_index_at_recruitment_f189_0_0, data = Lavaan_SEM, na.action=na.exclude)) }

for (i in c((length(Lavaan_SEM)-24):(length(Lavaan_SEM)-16))){
    
    Lavaan_SEM[[i]]=resid(lm(Lavaan_SEM[[i]] ~ age_when_attended_assessment_centre_f21003_0_0
                                 + average_total_household_income_before_tax_f738_0_0 + Sex
                                 + Depression + frequency_of_drinking_alcohol_20414.0.0
                                 + number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.0.0 
                                 + smoking_status_f20116_0_0 + qualifications_f6138_0_0 +
                                 + townsend_deprivation_index_at_recruitment_f189_0_0, data = Lavaan_SEM, na.action=na.exclude)) }

# Calculate characteristics

In [32]:
mean(Lavaan_SEM$age_when_attended_assessment_centre_f21003_2_0)
sd(Lavaan_SEM$age_when_attended_assessment_centre_f21003_2_0)
mean(Lavaan_SEM$body_mass_index_bmi_f21001_0_0, na.rm=TRUE)
sd(Lavaan_SEM$body_mass_index_bmi_f21001_0_0, na.rm=TRUE)
mean(Lavaan_SEM$WHR, na.rm=TRUE)
sd(Lavaan_SEM$WHR, na.rm=TRUE)
mean(Lavaan_SEM$body_fat_percentage_f23099_0_0, na.rm=TRUE)
sd(Lavaan_SEM$body_fat_percentage_f23099_0_0, na.rm=TRUE)
table(Lavaan_SEM$Sex)
table(Lavaan_SEM$Hypertension)
table(Lavaan_SEM$Diabetes)

## Measurement model
- Residual correlations between BMI and BFP based on https://pubmed.ncbi.nlm.nih.gov/23183515/

In [54]:
WMH_SEM = '
#Latent variables

Cognition =~ Mean_pairs + RT + Prospective_memory +
          FluidIntelligence + DigitSpan + Puzzles

Obesity =~ BMI + WHRscaled + BFP

Inflammation =~ CRP_scaled

Cerebrovascular =~ WMH_logscaled

Dyslipidemia =~ a*TG + a*HDL 

HT =~ Hypertension + SBP + DBP

DM =~ Diabetes + Glucose + HbA1c



#Constraints
WHRscaled ~~ Inflammation + HT + DM + Dyslipidemia


'


WMHOB_measure <- cfa(WMH_SEM, data=Lavaan_SEM, missing='pairwise', se='robust.sem', estimator='ML')
summary(WMHOB_measure, standardized=TRUE, fit.measures=TRUE)
#fitmeasures(WMHOB_fit) #fit indices

## Full model

In [16]:
WMH_SEM = '
#Latent variables

Cognition =~ Mean_pairs + RT + Prospective_memory +
          FluidIntelligence + DigitSpan + Puzzles

Obesity =~ BMI + WHRscaled + BFP

Inflammation =~ CRP_scaled

Cerebrovascular =~ WMH_logscaled

Dyslipidemia =~ a*TG + a*HDL #constrained because only two factors for a latent variable

HT =~  Hypertension + SBP + DBP

DM =~  Diabetes + Glucose + HbA1c

#regressions

HT + DM + Dyslipidemia + Inflammation ~
    Obesity 

Cerebrovascular ~ 
    Obesity + HT + DM + Dyslipidemia + Inflammation

MeanThickness + GM + GMsub ~ 
    Obesity + HT + DM + Dyslipidemia + Inflammation + Cerebrovascular

Cognition ~
    Obesity + HT + DM + Dyslipidemia + Inflammation + Cerebrovascular + MeanThickness + GM + GMsub

#covariance
WHRscaled ~~  HT + DM + Dyslipidemia
GM ~~ GMsub + MeanThickness
GMsub ~~ MeanThickness


'


WMHOB_GM_Thickness <- sem(WMH_SEM, data=Lavaan_SEM, missing='pairwise', se='robust.sem', estimator='ML')
summary(WMHOB_GM_Thickness, standardized=TRUE, fit.measures=TRUE)
#fitmeasures(WMHOB_fit) #fit indices

## Adjusted bonferroni correction of p-values according to https://www.tandfonline.com/doi/full/10.1080/10705511.2013.742385

In [17]:
cov=(lavInspect(WMHOB_GM_Thickness,"cor.all"))
diag(cov)=NA
alpha=0.05/(36^(1-(sqrt(mean(abs(cov),na.rm=T)))))
pvals=c(0,0,0,0,0.002,0,0,0.076,0.435,0.085,0.009,0,0.075,0.010,0,0,0,0.115,0.047,0.281,0,0,0,0.165,0,0.547,
       0,0.645,0.610,0.379,0.979,0.134,0,0.665,0.493,0.012)
pvals*(0.05/alpha)

In [18]:
lavInspect(WMHOB_GM_Thickness,'r2')

## Test linearity assumption

In [19]:
a=lavPredict(WMHOB_GM_Thickness, type='lv')
a=as.data.frame(a)
a=cbind(a,select(Lavaan_SEM, GM, GMsub, MeanThickness))


combs=(combn(c(1,2,3,4,5,6,7,8,9,10),2))
plot_list=list()

for (i in 1:ncol(combs)){

plot_list[[i]]=local({
        i <- i
        p1 <- ggplot(data=a, aes(a[[combs[,i][1]]],a[[combs[,i][2]]])) + 
geom_point() +
stat_smooth(method="loess") + 
stat_smooth(method="lm", color="red", fill="red", alpha=.25) +
xlab(colnames(a[combs[,i][1]])) + 
ylab(colnames(a[combs[,i][2]]))  })
    
    }

n <- length(plot_list)
nCol <- floor(sqrt(n))
pdf("/dagher/dagher11/filip/OBAL/output/Linearity.pdf", width = 24, height = 32) # Open a new pdf file
do.call("grid.arrange", c(plot_list, ncol=nCol))
dev.off()

## Test alternative models

In [20]:
WMH_SEM = '
#Latent variables

Cognition =~ Mean_pairs + RT + Prospective_memory +
          FluidIntelligence + DigitSpan + Puzzles

Obesity =~ BMI + WHRscaled + BFP

Inflammation =~ CRP_scaled

Dyslipidemia =~ a*TG + a*HDL #constrained because only two factors for a latent variable

HT =~  Hypertension + SBP + DBP

DM =~  Diabetes + Glucose + HbA1c

#regressions

HT + DM + Dyslipidemia + Inflammation ~
    Obesity 

Cognition ~
    Obesity + HT + DM + Dyslipidemia + Inflammation 

#covariance
WHRscaled ~~  HT + DM + Dyslipidemia
GM ~~ MeanThickness + GMsub
GMsub ~~ MeanThickness

'


WMHOB_NoBrain <- sem(WMH_SEM, data=Lavaan_SEM, missing='pairwise', se='robust.sem')
summary(WMHOB_NoBrain, standardized=TRUE, fit.measures=TRUE)
#fitmeasures(WMHOB_fit) #fit indices

In [21]:
WMH_SEM = '
#Latent variables

Cognition =~ Mean_pairs + RT + Prospective_memory +
          FluidIntelligence + DigitSpan + Puzzles

Obesity =~ BMI + WHRscaled + BFP

Inflammation =~ CRP_scaled

Dyslipidemia =~ a*TG + a*HDL #constrained because only two factors for a latent variable

HT =~  Hypertension + SBP + DBP

DM =~  Diabetes + Glucose + HbA1c

#regressions

HT + DM + Dyslipidemia + Inflammation ~
    Obesity 

MeanThickness + GM + GMsub ~ 
    Obesity + HT + DM + Dyslipidemia + Inflammation

Cognition ~
    Obesity + HT + DM + Dyslipidemia + Inflammation + MeanThickness + GM + GMsub

#covariance
WHRscaled ~~  HT + DM + Dyslipidemia
GM ~~ MeanThickness + GMsub
GMsub ~~ MeanThickness


'


WMHOB_noWMH <- sem(WMH_SEM, data=Lavaan_SEM, missing='pairwise', se='robust.sem')
summary(WMHOB_noWMH, standardized=TRUE, fit.measures=TRUE)
#fitmeasures(WMHOB_fit) #fit indices

In [22]:
WMH_SEM = '
#Latent variables

Cognition =~ Mean_pairs + RT + Prospective_memory +
          FluidIntelligence + DigitSpan + Puzzles

Obesity =~ BMI + WHRscaled + BFP

Inflammation =~ CRP_scaled

Cerebrovascular =~ WMH_logscaled

Dyslipidemia =~ a*TG + a*HDL #constrained because only two factors for a latent variable

HT =~  Hypertension + SBP + DBP

DM =~  Diabetes + Glucose + HbA1c

#regressions

HT + DM + Dyslipidemia + Inflammation ~
    Obesity 

Cerebrovascular ~ 
    Obesity + HT + DM + Dyslipidemia + Inflammation + MeanThickness + GM + GMsub

MeanThickness + GM + GMsub ~ 
    Obesity + HT + DM + Dyslipidemia + Inflammation 

Cognition ~
    Obesity + HT + DM + Dyslipidemia + Inflammation + Cerebrovascular + MeanThickness + GM + GMsub

#covariance
WHRscaled ~~  HT + DM + Dyslipidemia
GM ~~ MeanThickness + GMsub
GMsub ~~ MeanThickness


'


WMHOB_GM_toWMH <- sem(WMH_SEM, data=Lavaan_SEM, missing='pairwise', se='robust.sem')
summary(WMHOB_GM_toWMH, standardized=TRUE, fit.measures=TRUE)
#fitmeasures(WMHOB_fit) #fit indices

In [23]:
WMH_SEM = '
#Latent variables

Cognition =~ Mean_pairs + RT + Prospective_memory +
          FluidIntelligence + DigitSpan + Puzzles

Obesity =~ BMI + WHRscaled + BFP

Inflammation =~ CRP_scaled

Cerebrovascular =~ WMH_logscaled

Dyslipidemia =~ a*TG + a*HDL #constrained because only two factors for a latent variable

HT =~  Hypertension + SBP + DBP

DM =~  Diabetes + Glucose + HbA1c

#regressions

HT + DM + Dyslipidemia + Inflammation ~
    Obesity 

Cerebrovascular ~ 
    Obesity + HT + DM + Dyslipidemia + Inflammation 

Cognition ~
    Obesity + HT + DM + Dyslipidemia + Inflammation + Cerebrovascular 

#covariance
WHRscaled ~~   HT + DM + Dyslipidemia



'


WMHOB_noGM <- sem(WMH_SEM, data=Lavaan_SEM, missing='pairwise', se='robust.sem')
summary(WMHOB_noGM, standardized=TRUE, fit.measures=TRUE)
#fitmeasures(WMHOB_fit) #fit indices

In [24]:
for (i in 1:length(Lavaan_SEM)) {
    print(colnames(Lavaan_SEM[i]))
    print(sum(!is.na(Lavaan_SEM[i])))
}